In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv('C:\\Users\Administrator\Downloads\data\kdd99\kdddata_nodup.csv')
test_data = pd.read_csv('C:\\Users\Administrator\Downloads\data\kdd99\corrected', header=None)
print(len(train_data))
test_data.columns = train_data.columns

obj_col = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']
dos = ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']
u2r = ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']
r2l = ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']
probe = ['ipsweep', 'nmap', 'portsweep', 'satan']

x_train_obj = train_data[obj_col].astype('object', copy=False)
x_test_obj = test_data[obj_col].astype('object', copy=False)
x_train_fi = train_data.drop(columns=obj_col)
x_test_fi = test_data.drop(columns=obj_col)

1074992


In [3]:
# ohe = prep.LabelEncoder()
# for i in x_train_obj.columns:
#     x_train_obj[i] = ohe.fit_transform(x_train_obj[i])
# x_train = pd.concat([x_train_obj, x_train_fi], axis=1)
# x_train_obj

ohe = prep.OneHotEncoder(handle_unknown='ignore')
x_train_obj = pd.DataFrame(ohe.fit_transform(x_train_obj).toarray())
x_test_obj = pd.DataFrame(ohe.transform(x_test_obj).toarray())
x_train = pd.concat([x_train_obj, x_train_fi], axis=1)
x_test = pd.concat([x_test_obj, x_test_fi], axis=1)

In [4]:
def func(s):
    if s in dos:
        return 'dos'
    elif s in u2r:
        return 'u2r'
    elif s in r2l:
        return 'r2l'
    elif s in probe:
        return 'probe'
    else:
        return s


x_train['class'] = x_train['class'].apply(func)
x_train['class'].value_counts()

x_train_nor = x_train[x_train['class'] == 'normal']
x_train_dos = x_train[x_train['class'] == 'dos']
x_train_pro = x_train[x_train['class'] == 'probe']
x_train_r2l = x_train[x_train['class'] == 'r2l']
x_train_u2r = x_train[x_train['class'] == 'u2r']

In [5]:
print(x_train.shape)
x_train["class"].value_counts()
print(x_test.shape)
x_test["class"].value_counts()

(1074992, 127)
(311029, 127)


smurf.              164091
normal.              60593
neptune.             58001
snmpgetattack.        7741
mailbomb.             5000
guess_passwd.         4367
snmpguess.            2406
satan.                1633
warezmaster.          1602
back.                 1098
mscan.                1053
apache2.               794
processtable.          759
saint.                 736
portsweep.             354
ipsweep.               306
httptunnel.            158
pod.                    87
nmap.                   84
buffer_overflow.        22
multihop.               18
named.                  17
sendmail.               17
ps.                     16
rootkit.                13
xterm.                  13
teardrop.               12
xlock.                   9
land.                    9
xsnoop.                  4
ftp_write.               3
sqlattack.               2
perl.                    2
worm.                    2
loadmodule.              2
udpstorm.                2
phf.                     2
i

In [6]:
def mk_dup(s):
    while s.index.size < 100000:
        s = s.append(s)
    return s


x_train_pro = mk_dup(x_train_pro)
x_train_r2l = mk_dup(x_train_r2l)
x_train_u2r = mk_dup(x_train_u2r)

In [7]:
x_train = pd.concat([x_train_nor, x_train_dos, x_train_pro, x_train_u2r, x_train_r2l])
y_train = x_train['class']
x_train = x_train.drop(columns=["class"])
y_test = x_test['class']
x_test = x_test.drop(columns=["class"])
print(y_train.shape, y_train.value_counts())
print(y_test.shape, y_test.value_counts())

(1405329,) normal    812814
dos       247267
r2l       127872
probe     110880
u2r       106496
Name: class, dtype: int64
(311029,) smurf.              164091
normal.              60593
neptune.             58001
snmpgetattack.        7741
mailbomb.             5000
guess_passwd.         4367
snmpguess.            2406
satan.                1633
warezmaster.          1602
back.                 1098
mscan.                1053
apache2.               794
processtable.          759
saint.                 736
portsweep.             354
ipsweep.               306
httptunnel.            158
pod.                    87
nmap.                   84
buffer_overflow.        22
multihop.               18
named.                  17
sendmail.               17
ps.                     16
rootkit.                13
xterm.                  13
teardrop.               12
xlock.                   9
land.                    9
xsnoop.                  4
ftp_write.               3
sqlattack.               2
perl

In [8]:
np.save("kdd99_oh", x_train)
np.save("kdd99_oh_tar", y_train)
np.save("kdd99_oh_corrected", x_test)
np.save("kdd99_oh_tar_corrected", y_test)
print(x_train.shape,x_test.shape)

(1405329, 126) (311029, 126)
